In [1]:
import pandas as pd

name_of_csv_file = r'C:\Users\wipa0001\Desktop\CREF_EXCEPTIONS_ES_2153.txt' #Update the input file path
Current_period = 2147 #Provide the current period
Range = 5 #Recommended 5
Mean_multiplier = 10 #Recommended 10
MaxMin_multiplier = 5 #Recommended 5

df = pd.read_table(f'{name_of_csv_file}', sep=';', dtype ={'Cref' : 'str'},
                  usecols=['Period','Ref period','Cref',
                           'Sales value','Sales Units','Price'])

df = df[df['Cref'] != '0000000000000'] #Removing crefs 0000000000000

group = df.groupby('Cref')

#Range
quantile_25 = group.quantile(q=.25)
quantile_25.reset_index(inplace = True)
quantile_25 = quantile_25[['Cref', 'Sales value']]
quantile_25.set_index('Cref', inplace = True)

quantile_75 = group.quantile(q=.75)
quantile_75.reset_index(inplace = True)
quantile_75 = quantile_75[['Cref', 'Sales value']]
quantile_75.set_index('Cref', inplace = True)

dfTotal = quantile_25.join(quantile_75, how = 'inner', lsuffix = '_25', rsuffix = '_75')
dfTotal['Range_Sales value'] = dfTotal['Sales value_75'] - dfTotal['Sales value_25']

#Median
df_median = group.median()
df_median.reset_index(inplace = True)
df_median = df_median[['Cref', 'Sales value']]
df_median.rename(columns = {'Sales value' : 'Median Sales value'}, inplace = True)

dfTotal = dfTotal.merge(df_median, on = 'Cref')
dfTotal[f'Median + {str(Range)} x Range'] = dfTotal['Median Sales value'] + dfTotal['Range_Sales value']*Range
dfTotal[f'{str(Mean_multiplier)} x Median'] = dfTotal['Median Sales value'] * Mean_multiplier

#Max_Min
Max_Min = df.pivot_table(index = 'Cref', values = 'Price', aggfunc = ['max', 'min'])
Max_Min.columns = Max_Min.columns.droplevel(level = 1)
Max_Min['Max-Min'] = Max_Min['max'] - Max_Min['min']
Max_Min.reset_index(inplace = True)

dfTotal = dfTotal.merge(Max_Min, on = 'Cref')

#Merging
df_Current_period = df[df['Ref period'] == Current_period]
dfFinal = df_Current_period.merge(dfTotal, on = 'Cref')

#Checks calculations
dfFinal[f'Sales Value - Median + {str(Range)} x Range'] = dfFinal['Sales value'] - dfFinal[f'Median + {str(Range)} x Range']
dfFinal[f'Sales Value - {str(Mean_multiplier)}x Median'] = dfFinal['Sales value'] - dfFinal[f'{str(Mean_multiplier)} x Median']

def outlier(row):
    if (row['Max-Min']) > MaxMin_multiplier * row['min']:  
        return 'o'
    else:
        return ' '
    
dfFinal['Outlier check'] = dfFinal.apply(outlier, axis = 1)

#Filters for finding outliers
filter1 = dfFinal[f'Sales Value - Median + {str(Range)} x Range'] > 0
filter2 = dfFinal[f'Sales Value - {str(Mean_multiplier)}x Median'] > 0
filter3 = dfFinal['Outlier check'] == 'o'

dfFiltered = dfFinal.where(filter1 | filter2 | filter3).dropna(how = 'all')
dfFiltered

# You will get potential outliers 

,Period,Ref period,Cref,Sales value,Sales Units,Price,Sales value_25,Sales value_75,Range_Sales value,Median Sales value,Median + 5 x Range,10 x Median,max,min,Max-Min,Sales Value - Median + 5 x Range,Sales Value - 10x Median,Outlier check
0,2153.0,2147.0,0000002000013,501.07,207.0,2.42,297.3325,473.7875,176.455,390.205,1272.480,3902.05,622.78,1.94,620.84,-771.410,-3400.98,o
16,2153.0,2147.0,0000087170160,23.46,3.0,7.82,23.5075,158.8525,135.345,82.040,758.765,820.40,19.40,1.15,18.25,-735.305,-796.94,o
17,2153.0,2147.0,0000087170160,167.04,9.0,18.56,23.5075,158.8525,135.345,82.040,758.765,820.40,19.40,1.15,18.25,-591.725,-653.36,o
18,2153.0,2147.0,0000087170160,71.03,5.0,14.21,23.5075,158.8525,135.345,82.040,758.765,820.40,19.40,1.15,18.25,-687.735,-749.37,o
19,2153.0,2147.0,0000087170160,171.23,31.0,5.52,23.5075,158.8525,135.345,82.040,758.765,820.40,19.40,1.15,18.25,-587.535,-649.17,o


In [2]:
potential_outliers = dfFiltered['Cref'].unique()

df_orginal = pd.read_table(f'{name_of_csv_file}',sep =';', dtype ={'Cref' : 'str'})
df_orginal[df_orginal['Cref'].isin(potential_outliers)]

#Verify by checking potentail outliers with original file

,Period,Ref period,Store Id,Cref,Cref Suffix,Cref Description,Hash Signature,Store incr. on past period(%),Cref contrib. to total store(%),Store Increment PCT,Cref Contrib PCT,Sales value,Sales Units,Price
0,2153,2153,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,93.6396,48.1095,40,40,69128.05,111.0,622.78
1,2153,2152,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,-6.1248,0.2173,40,40,161.28,83.0,1.94
2,2153,2151,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,-1.1638,0.3003,40,40,237.37,110.0,2.16
3,2153,2150,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,-3.5916,0.4901,40,40,391.94,177.0,2.21
4,2153,2149,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,-2.8629,0.4688,40,40,388.91,163.0,2.39
5,2153,2148,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,1.7437,0.7327,40,40,625.71,246.0,2.54
6,2153,2147,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,-1.3727,0.5970,40,40,501.07,207.0,2.42
7,2153,2146,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,2.4949,0.3303,40,40,281.07,141.0,1.99
8,2153,2145,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,-6.047,0.4168,40,40,346.12,162.0,2.14
9,2153,2144,1600000957,0000002000013,D:3,FRUTAS Y VERDURAS 1 UNI,1283382460,-,0.4430,40,40,391.50,197.0,1.99


In [3]:
a = df_orginal[df_orginal['Cref'].isin(potential_outliers)]


In [4]:
a.to_excel('cref_report_outlier_2150.xlsx')